# Disease

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Mondo](https://mondo.monarchinitiative.org/)
2. [Human Disease](https://disease-ontology.org/)

Here we show how to access and search disease ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-disease --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## Bionty objects

Let us create a public knowledge accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.bionty`, which chooses a default public knowledge source from {class}`~docs:lnschema_bionty.PublicSource`. It's a [Bionty](https://lamin.ai/docs/bionty/bionty.bionty) object, which you can think about as a less-capable registry:

In [ ]:
public = lb.Disease.public()
public

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = public.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = public.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.alzheimer_disease

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["Alzheimer disease"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = public.lookup(public.ontology_id)

In [ ]:
lookup.mondo_0004975

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
public.search("parkinsons disease").head(3)

By default, search also covers synonyms:

In [ ]:
public.search("paralysis agitans").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
public.search("parkinsons disease", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
public.search(
    "progressive degenerative disorder of the central nervous system",
    field=public.definition,
).head()

## Standardize disease identifiers

Let us generate a `DataFrame` that stores a number of disease identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "supraglottis cancer",
        "alexia",
        "trigonitis",
        "paranasal sinus disorder",
        "This disease does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = public.validate(df_orig.index, public.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Disease").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="mondo", version="2023-04-04", organism="all"
).one()
public = lb.Disease.public(public_source=public_source)
public

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-disease
!rm -r test-disease